In [12]:
import pprint as pp
import requests
import pandas as pd
import requests
import plotly.graph_objects as go

In [13]:
def get_latest_events():
    try:
        response = requests.get("https://eonet.gsfc.nasa.gov/api/v3/events")
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def preprocess_wildfires_df(events):
    events = get_latest_events()
    if events:
        # Extract relevant information from each event and add it to a list
        data = []
        for event in events["events"]:
            data.append(
                {
                    "Event": event["title"],
                    "Categories": event["categories"][0]["title"]
                    if event["categories"]
                    else None,
                    "Date": event["geometry"][0]["date"] if event["geometry"] else None,
                    "Location": event["geometry"][0]["coordinates"]
                    if event["geometry"]
                    else None,
                }
            )
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(data)
        df["Country"] = df["Event"].apply(
            lambda event: "United States"
            if "States" in event
            else event.split(",")[-1].strip()
        )
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        df["Season"] = df["Date"].dt.to_period("Q").astype(str)

        df[["Longitude", "Latitude"]] = pd.DataFrame(
            df["Location"].tolist(), index=df.index
        )

    df = df[(df["Country"] == "United States") & (df["Categories"] == "Wildfires")]
    seasons = df["Season"].unique()
    new_seasons = {season: season[:4] + " " + season[4:] for season in seasons}
    df["Season"] = df["Season"].replace(new_seasons)

    return df

In [14]:
events = get_latest_events()
df = preprocess_wildfires_df(events)

C:\Users\redhawc\AppData\Local\Temp\ipykernel_23300\2294503821.py:41: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [15]:
df.head()

,Event,Categories,Date,Location,Country,Year,Season,Longitude,Latitude
22,"Major Fire, Ocala National Forest, Florida, Un...",Wildfires,2023-04-27 14:15:00+00:00,"[-81.688889, 29.229722]",United States,2023,2023 Q2,-81.688889,29.229722
55,"Garceau Fire, W of Polson, Montana, United States",Wildfires,2022-08-16 00:00:00+00:00,"[-114.489, 47.694]",United States,2022,2022 Q3,-114.489000,47.694000
105,"Wildfire - N of Merced, California - United St...",Wildfires,2022-02-16 20:44:00+00:00,"[-120.533005422, 37.543469882]",United States,2022,2022 Q1,-120.533005,37.543470
106,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-02-13 20:40:00+00:00,"[-110.862229905, 33.417091555]",United States,2022,2022 Q1,-110.862230,33.417092
111,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-01-28 08:38:00+00:00,"[-110.860071422, 33.416654251]",United States,2022,2022 Q1,-110.860071,33.416654


In [16]:
!pip install plotly geopandas pyshp shapely


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
temperature = pd.read_csv("https://raw.githubusercontent.com/REDhawC/EDI_BUSANA/master/Python_programming/coding_problems/TemperatureByStates_Updated.csv")
precipitation = pd.read_csv("https://raw.githubusercontent.com/REDhawC/EDI_BUSANA/master/Python_programming/coding_problems/PrecipitationByStates_Updated.csv")

In [18]:
temperature.head()

,State,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3,2023 Q4,code,state
0,Alabama,50.966667,69.333333,77.733333,58.766667,50.433333,72.133333,78.300000,55.266667,55.533333,70.133333,79.800000,65.2,AL,Alabama
1,Alaska,6.900000,38.366667,47.566667,13.100000,9.633333,39.233333,48.800000,17.500000,10.166667,35.466667,50.000000,27.8,AK,Alaska
2,Arizona,46.166667,69.900000,78.600000,53.766667,46.566667,69.366667,78.733333,50.166667,43.166667,65.766667,79.900000,64.6,AZ,Arizona
3,Arkansas,43.800000,67.366667,78.100000,56.100000,44.366667,69.633333,79.100000,51.500000,49.366667,69.200000,79.166667,64.1,AR,Arkansas
4,California,47.033333,65.866667,76.333333,52.166667,49.233333,63.433333,76.500000,51.266667,43.200000,61.333333,74.233333,62.1,CA,California


In [19]:
print(df["Season"].unique())

['2023 Q2' '2022 Q3' '2022 Q1' '2021 Q4' '2021 Q3']


In [20]:
# 假设 temperature 是您的温度数据集
temperature["text"] = temperature["state"] + "<br>"

# 创建图表
fig = go.Figure()

# 添加初始季度的温度数据轨迹
temperature_trace = go.Choropleth(
    visible=True,  # 保证初始季度的温度数据可见
    locations=temperature["code"],
    z=temperature["2021 Q3"].astype(float),  # 初始显示2021Q3
    locationmode="USA-states",
    colorscale="Reds",
    name="Temperature",
    autocolorscale=False,
    marker_line_color="white",  # line markers between states
    colorbar_title="↑ Temperature",
)
fig.add_trace(temperature_trace)

# 使用气泡图来显示降水量
precipitation_trace = go.Scattergeo(
    visible=True,
    locations=precipitation["code"],  # 使用州代码作为位置标识
    locationmode="USA-states",
    marker=dict(
        size=precipitation["2021 Q3"] * 2,  # 使用降水量数据来决定气泡大小
        color="skyblue",
        opacity=1,
        line=dict(width=0),
    ),
    name="Precipitation",
)
fig.add_trace(precipitation_trace)

# 假设 wildfires 是您的野火数据集
# 为每个季度添加野火数据轨迹
seasons = [
    "2021 Q1",
    "2021 Q2",
    "2021 Q3",
    "2021 Q4",
    "2022 Q1",
    "2022 Q2",
    "2022 Q3",
    "2022 Q4",
    "2023 Q1",
    "2023 Q2",
    "2023 Q3",
    "2023 Q4",
]
for season in seasons:
    season_df = df[df["Season"] == season]
    fig.add_trace(
        go.Scattergeo(
            visible=(season == "2021 Q3"),  # 仅对初始季度设为可见
            lon=season_df["Longitude"],
            lat=season_df["Latitude"],
            text=["🔥" for _ in range(len(season_df))],  # 使用火焰 emoji
            mode="text",  # 将模式设置为"text"以显示文本标记
            name="Wildfire",
            textfont=dict(
                family="Arial",  # 可以选择合适的字体
                size=12,  # 调整大小以适合你的图表
                color="red",  # 文本的颜色，这里是红色
            ),
        )
    )

# 更新布局
fig.update_layout(
    title_text="2021-2023 US Wildfire Cases Visualisation<br>Compared With Temperature & Precipitation By State",
    geo=dict(
        scope="usa",
        projection=go.layout.geo.Projection(type="albers usa"),
        showlakes=True,  # lakes
        lakecolor="rgb(255, 255, 255)",
    ),
    showlegend=False,
)


# 添加滑动条以切换季度
steps = []
for index1, season in enumerate(seasons):
    step = dict(
        method="update",
        args=[
            {
                "z": [temperature[season].astype(float)],  # 更新温度数据
                "marker.size": [precipitation[season] * 2],  # 更新降水量数据的气泡大小
                "visible": [True, True]  # 保持温度和降水量数据可见
                + [index1 == index2 for index2 in range(len(seasons))],  # 更新野火数据的可见性
            },
        ],
        label=season,
    )
    steps.append(step)

slider = dict(active=2, currentvalue={"prefix": "Season: "}, steps=steps)

fig.update_layout(sliders=[slider])

# 添加注释
fig.add_annotation(
    x=0.5,
    y=-0.15,
    xref="paper",
    yref="paper",
    text="The bigger the bubble is, the more precipitation in this region.",
    showarrow=False,
    font=dict(size=12, color="purple"),
    align="center",
)

# 显示图表
fig.show()

In [21]:
# visualisation(temperture, Precipitation, df)